In [126]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings

### Fetching Lyrics from Genius 

In [ ]:
token = open('token.txt','r').readline()

In [127]:
def fetch_song_info(artist,title,token):
    genius_api_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + token}
    search_url = genius_api_url + '/search'
    query = {'q':title + '' + artist}
    response = requests.get(search_url,data=query,headers=headers)
    search_hits = response.json()['response']['hits']
    song_path = ''
    
    for hit in search_hits:
        a = hit['result']['primary_artist']['name']
        t = hit['result']['title'] 
        
        if a == artist and t == title:
            song_path = hit['result']['path']
            break
    
    if len(song_path) == 0:
        warnings.warn('No matching result found')
    
    return response,song_path

In [ ]:
def scrape_lyrics(path):
    genius_web = 'https://genius.com/'
    page = requests.get(genius_web+song_path).content
    soup = bs(page,'html.parser')
    lyrics_content = soup.find('div',class_='lyrics').findAll('a')
    lyrics = []
    
    for i in range(len(lyrics_content)):
        lyrics.append(lyrics_content[i].get_text())
    
    return lyrics

In [ ]:
response,song_path = fetch_song_info('Radiohead','true love waits',token)

In [ ]:
lyrics = scrape_lyrics(song_path)

In [ ]:
for verse in lyrics:
    print(verse)

### Billboard Hot 100 Songs 

In [ ]:
def get_hot_100(date):
    """Function to scrape the Billboard Hot 100 tracks
       
       Parameters:
       -----------
       date(string): date to fetch data for. should be of YYYY-MM-DD format
       
       Returns:
       --------
       pandas dataframe containing the artist names and track titles
    
    """
    
    url = 'https://www.billboard.com/charts/hot-100/' + date
    response = requests.get(url)
    hot_soup = bs(response.content,'html.parser')
    
    no1_artist = hot_soup.find('div',class_="chart-number-one__artist").text.split('\n')[2]
    no1_title = hot_soup.find('div',class_="chart-number-one__title").text
    
    artists = [item["data-artist"] for item in hot_soup.find_all('div',class_="chart-list-item") if "data-artist" in item.attrs]
    titles = [item["data-title"] for item in hot_soup.find_all('div',class_="chart-list-item") if "data-title" in item.attrs]
    
    artists = [no1_artist] + artists
    titles = [no1_title] + titles
    
    hot100 = pd.DataFrame({'Artist':artists,'Title':titles})
    
    return hot100

In [129]:
get_hot_100('2018-01-01').head()

,Artist,Title
0,Ed Sheeran,Perfect
1,Post Malone Featuring 21 Savage,Rockstar
2,Camila Cabello Featuring Young Thug,Havana
3,Lil Pump,Gucci Gang
4,G-Eazy Featuring A$AP Rocky & Cardi B,No Limit
